In [ ]:
!wget https://bootstrap.pypa.io/get-pip.py



--2025-05-20 04:06:23--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2279307 (2.2M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.17M  --.-KB/s    in 0.07s   

2025-05-20 04:06:23 (29.4 MB/s) - ‘get-pip.py’ saved [2279307/2279307]



In [ ]:
!python3.10 get-pip.py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pip]


In [ ]:
!python --version


Python 3.11.12


In [1]:
from google.colab import files
uploaded = files.upload()  # Choose your .zip or .py files


Saving UGBench.zip to UGBench.zip


In [2]:
!unzip UGBench.zip -d Unlearn_Harry

Archive:  UGBench.zip
   creating: Unlearn_Harry/UGBench/
   creating: Unlearn_Harry/UGBench/.git/
  inflating: Unlearn_Harry/UGBench/.git/config  
  inflating: Unlearn_Harry/UGBench/.git/description  
  inflating: Unlearn_Harry/UGBench/.git/HEAD  
   creating: Unlearn_Harry/UGBench/.git/hooks/
  inflating: Unlearn_Harry/UGBench/.git/hooks/applypatch-msg.sample  
  inflating: Unlearn_Harry/UGBench/.git/hooks/commit-msg.sample  
  inflating: Unlearn_Harry/UGBench/.git/hooks/fsmonitor-watchman.sample  
  inflating: Unlearn_Harry/UGBench/.git/hooks/post-update.sample  
  inflating: Unlearn_Harry/UGBench/.git/hooks/pre-applypatch.sample  
  inflating: Unlearn_Harry/UGBench/.git/hooks/pre-commit.sample  
  inflating: Unlearn_Harry/UGBench/.git/hooks/pre-merge-commit.sample  
  inflating: Unlearn_Harry/UGBench/.git/hooks/pre-push.sample  
  inflating: Unlearn_Harry/UGBench/.git/hooks/pre-rebase.sample  
  inflating: Unlearn_Harry/UGBench/.git/hooks/pre-receive.sample  
  inflating: Unlearn_H

In [1]:
%cd /content/Unlearn_Harry/Harry

/content/Unlearn_Harry/Harry


In [2]:
!pwd

/content/Unlearn_Harry/Harry


In [3]:
import sys
sys.path.append('/content/Unlearn_Harry/Harry')


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.6/836.6 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 

In [17]:
%%writefile scripts/run_unlearning.sh

#!/bin/bash
# Exit script on any error
set -e

echo "===== Starting Harry Potter unlearning experiment ====="

# Setup environment variables
export dataset="Harry"
export MASTER_PORT=18765
export split=forget
export model=deepseek-r1-distill
export num_epochs=5
export batch_size=1
export gradaccum=2
export cache=$PWD/cache
export forget_data_path=$PWD/data/${dataset}
export CUDA_VISIBLE_DEVICES=0  # Using first GPU
export retain_weight=1
export lr=2e-5
export forget_loss="grad_ascent+kl"
export save_dir=$PWD/experiment/${dataset}/${model}/${forget_loss}_E${num_epochs}_B${batch_size}_G${gradaccum}_lr${lr}_W${retain_weight}

# Create directories if needed
echo "Creating necessary directories..."
mkdir -p $cache
mkdir -p $save_dir/eval_results

# Print current configuration
echo "===== Configuration ====="
echo "Dataset: $dataset"
echo "Model: $model"
echo "Forget method: $forget_loss"
echo "Save directory: $save_dir"
echo "======================="

# Step 1: Run the forget step
echo "===== Step 1: Running the forget step ====="
if [[ ${forget_loss} != "icl" ]]; then
    echo "Running forget.py..."
    PYTHONPATH=$PYTHONPATH:$PWD python forget.py --config-name=forget.yaml \
        dataset=$dataset split=${split} \
        forget_data_path=${forget_data_path} \
        retain_data_path=${forget_data_path} \
        forget_loss=${forget_loss} batch_size=${batch_size} \
        retain_weight=${retain_weight} \
        gradient_accumulation_steps=${gradaccum} model_family=${model} lr=${lr} \
        save_dir=$save_dir cache_dir=$cache num_epochs=${num_epochs}

    echo "Forget step completed successfully."
else
    echo "Skipping forget step for ICL method."
fi

# Step 2: Run the evaluation
echo "===== Step 2: Running evaluation ====="
echo "Running evaluate_${dataset}.py..."
PYTHONPATH=$PYTHONPATH:$PWD python evaluate_${dataset}.py \
    model_family=$model dataset=${dataset} \
    split=${split} batch_size=4 \
    model_path=$save_dir \
    generation.max_length=200 forget_loss=${forget_loss} \
    save_dir=$save_dir/eval_results

echo "Evaluation completed successfully."

# Step 3: Aggregate statistics
echo "===== Step 3: Aggregating statistics ====="
echo "Running aggregate_eval_stat.py..."
PYTHONPATH=$PYTHONPATH:$PWD python aggregate_eval_stat.py \
    ckpt_result=$save_dir/eval_results/eval_log_aggregated.json \
    method_name=$forget_loss \
    save_file=$save_dir/eval_results/eval.csv \
    excel_file_path=$save_dir/eval_results/eval.xlsx \
    submitted_by=who

echo "===== All steps completed successfully! ====="
echo "Results saved to: $save_dir/eval_results/"

Writing scripts/run_unlearning.sh


In [18]:
!chmod +x scripts/run_unlearning.sh

In [ ]:
!bash scripts/test.sh

===== Fixing Hugging Face dependencies =====
Current package versions:
datasets                              2.18.0
huggingface-hub                       0.17.3
sentence-transformers                 4.1.0
tensorflow-datasets                   4.9.8
transformers                          4.35.0
vega-datasets                         0.9.0
Installing compatible package versions...
Found existing installation: huggingface-hub 0.17.3
Uninstalling huggingface-hub-0.17.3:
  Successfully uninstalled huggingface-hub-0.17.3
Found existing installation: datasets 2.18.0
Uninstalling datasets-2.18.0:
  Successfully uninstalled datasets-2.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
evaluate 0.4.2 requires datasets>=2.0.0, which is not installed.
torchtune 0.6.1 requires datasets, which is

In [ ]:
!pip install --upgrade pip

!pip install \
    transformers==4.39.3 \
    datasets==2.18.0 \
    huggingface-hub==0.20.3 \
    bitsandbytes==0.43.1 \
    peft==0.10.0 \
    triton==2.0.0 \
    sentence-transformers==2.2.2 \
    accelerate==0.29.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 112.3 MB/s eta 0:00:00
  DEPRECATION: Building 'sentence-transformers' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sentence-transformers'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl s

In [7]:
!pip install numpy==1.25.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 112.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.29.0 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.0.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.31.0 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.10.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.2 which is incompatible.
thinc 8.3.6 

In [ ]:
!bash scripts/run_unlearning.sh

===== Starting Harry Potter unlearning experiment =====
Creating necessary directories...
===== Configuration =====
Dataset: Harry
Model: phi
Forget method: grad_ascent+kl
Save directory: /content/Unlearn_Harry/Harry/experiment/Harry/phi/grad_ascent+kl_E5_B2_G4_lr2e-5_W1
===== Step 1: Running the forget step =====
Running forget.py...
2025-05-20 07:09:56.631344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747724996.654599   14736 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747724996.661365   14736 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[2025-05-20 07:10:00,523] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to

In [4]:
%%writefile scripts/finetune.sh

#!/bin/bash
export dataset="Harry";   # [TOFU, Harry, ZSRE]
export master_port=18765;
export model=deepseek-r1-distill;   # [phi, llama2-7b]
export split=finetune;
export data_path=$PWD/data/${dataset}/${split}.json;
export lr=3e-5;
export batch_size=1;
export GA=2;
export epoch=5;
export save_file=$PWD/save_model/${dataset}/${split}_${model}_B${batch_size}_G${GA}_E${epoch}_lr${lr};
export CUDA_VISIBLE_DEVICES=0;

deepspeed finetune.py --config-name finetune.yaml \
    dataset=${dataset} \
    data_path=${data_path} \
    split=${split} \
    batch_size=${batch_size} \
    gradient_accumulation_steps=${GA} \
    num_epochs=${epoch} \
    lr=${lr} \
    model_family=${model} \
    save_dir=${save_file} \
    weight_decay=0.01 \
    bf16=true

if [[ ${dataset} = "TOFU" ]];
then
declare -A retain_splits;
retain_splits["retain99"]="forget01";
retain_splits["retain95"]="forget05";
retain_splits["retain90"]="forget10";

python evaluate_${dataset}.py \
    model_family=$model dataset=${dataset} \
    split=${retain_splits[${split}]} \
    model_path=$save_file batch_size=${batch_size} \
    generation.max_length=200 \
    save_dir=$save_file/eval_results;

python aggregate_eval_stat.py \
    ckpt_result=$save_file/eval_results/eval_log_aggregated.json \
    method_name="finetune" \
    save_file=$save_file/eval_results/eval.csv \
    excel_file_path=$save_file/eval_results/eval.xlsx \
    submitted_by=who;

elif [[ ${dataset} = "Harry" ]];
then
python evaluate_${dataset}.py \
    model_family=$model dataset=${dataset} \
    split=forget_all_subject batch_size=${batch_size}\
    model_path=$save_file \
    generation.max_length=200 \
    save_dir=$save_file/eval_results;

python aggregate_eval_stat.py \
    ckpt_result=$save_file/eval_results/eval_log_aggregated.json \
    method_name="finetune" \
    save_file=$save_file/eval_results/eval.csv \
    excel_file_path=$save_file/eval_results/eval.xlsx \
    submitted_by=who;

elif [[ ${dataset} = "ZSRE" ]];
then
export Types=(inverse onehop subject_replace);
for type in "${Types[@]}"
do
    python evaluate_${dataset}.py --config-name eval_${type}.yaml \
        model_family=$model dataset=${dataset} \
        split=forget batch_size=${batch_size} \
        model_path=$save_file \
        generation.max_length=200 \
        save_dir=$save_file/eval_${type}_results; \

    python aggregate_eval_stat.py \
        ckpt_result=$save_file/eval_${type}_results/eval_log_aggregated.json \
        method_name="finetune" \
        save_file=$save_file/eval_${type}_results/eval.csv \
        excel_file_path=$save_file/eval_${type}_results/eval.xlsx \
        submitted_by=who;
done
fi


Overwriting scripts/finetune.sh


In [5]:
!chmod +x scripts/finetune.sh

In [ ]:
!pip uninstall -y bitsandbytes


Found existing installation: bitsandbytes 0.43.1
Uninstalling bitsandbytes-0.43.1:
  Successfully uninstalled bitsandbytes-0.43.1


In [ ]:
!pip uninstall -y bitsandbytes
!pip install bitsandbytes-cuda117  # or cuda118 based on your runtime
!pip install accelerate --upgrade
!pip install deepspeed --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.2
    Uninstalling accelerate-0.29.2:
      Successfully uninstalled accelerate-0.29.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [accelerate]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'deepspeed' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of

In [6]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"


In [7]:
!nvidia-smi

Tue May 20 08:31:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [7]:
!bash scripts/finetune.sh

[2025-05-21 06:10:52,633] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-05-21 06:10:55,029] [WARNING] [runner.py:196:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
Detected CUDA_VISIBLE_DEVICES=0: setting --include=localhost:0
[2025-05-21 06:10:55,046] [INFO] [runner.py:555:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None finetune.py --config-name finetune.yaml dataset=Harry data_path=/content/Unlearn_Harry/Harry/data/Harry/finetune.json split=finetune batch_size=1 gradient_accumulation_steps=2 num_epochs=3 lr=3e-5 model_family=deepseek-r1-distill save_dir=/content/Unlearn_Harry/Harry/save_model/Harry/finetune_deepseek-r1-distill_B1_G2_E3_lr3e-5 weight_decay=0.01 bf16=true
[2025-05-21 06:10:57,098] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelera

In [8]:
%%writefile scripts/eval.sh

#!/bin/bash
export dataset="Harry";   # [TOFU, Harry, ZSRE]
export master_port=18765;
export model=deepseek-r1-distill;   # [phi, llama2-7b]
export split=finetune;
export data_path=$PWD/data/${dataset}/${split}.json;
export lr=3e-5;
export batch_size=1;
export GA=2;
export epoch=3;
export save_file=$PWD/save_model/${dataset}/${split}_${model}_B${batch_size}_G${GA}_E${epoch}_lr${lr};
export CUDA_VISIBLE_DEVICES=0;

if [[ ${dataset} = "TOFU" ]];
then
declare -A retain_splits;
retain_splits["retain99"]="forget01";
retain_splits["retain95"]="forget05";
retain_splits["retain90"]="forget10";

python evaluate_${dataset}.py \
    model_family=$model dataset=${dataset} \
    split=${retain_splits[${split}]} \
    model_path=$save_file batch_size=${batch_size} \
    generation.max_length=200 \
    save_dir=$save_file/eval_results;

python aggregate_eval_stat.py \
    ckpt_result=$save_file/eval_results/eval_log_aggregated.json \
    method_name="finetune" \
    save_file=$save_file/eval_results/eval.csv \
    excel_file_path=$save_file/eval_results/eval.xlsx \
    submitted_by=SameedandTaha;

elif [[ ${dataset} = "Harry" ]];
then
python evaluate_${dataset}.py \
    model_family=$model dataset=${dataset} \
    split=forget batch_size=${batch_size}\
    model_path=$save_file \
    generation.max_length=200 \
    save_dir=$save_file/eval_results;

python aggregate_eval_stat.py \
    ckpt_result=$save_file/eval_results/eval_log_aggregated.json \
    method_name="finetune" \
    save_file=$save_file/eval_results/eval.csv \
    excel_file_path=$save_file/eval_results/eval.xlsx \
    submitted_by=SameedandTaha;

elif [[ ${dataset} = "ZSRE" ]];
then
export Types=(inverse onehop subject_replace);
for type in "${Types[@]}"
do
    python evaluate_${dataset}.py --config-name eval_${type}.yaml \
        model_family=$model dataset=${dataset} \
        split=forget batch_size=${batch_size} \
        model_path=$save_file \
        generation.max_length=200 \
        save_dir=$save_file/eval_${type}_results; \

    python aggregate_eval_stat.py \
        ckpt_result=$save_file/eval_${type}_results/eval_log_aggregated.json \
        method_name="finetune" \
        save_file=$save_file/eval_${type}_results/eval.csv \
        excel_file_path=$save_file/eval_${type}_results/eval.xlsx \
        submitted_by=who;
done
fi


Writing scripts/eval.sh


In [9]:
!chmod +x scripts/eval.sh

In [22]:
!bash scripts/eval.sh

2025-05-21 08:29:46.882494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747816186.904238   47204 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747816186.910912   47204 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the voca

In [40]:
!bash scripts/run_unlearning.sh

===== Starting Harry Potter unlearning experiment =====
Creating necessary directories...
===== Configuration =====
Dataset: Harry
Model: deepseek-r1-distill
Forget method: grad_ascent+kl
Save directory: /content/Unlearn_Harry/Harry/experiment/Harry/deepseek-r1-distill/grad_ascent+kl_E1_B2_G4_lr2e-5_W1
===== Step 1: Running the forget step =====
Running forget.py...
2025-05-21 11:03:29.227114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747825409.249521   87247 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747825409.256322   87247 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[2025-05-21 11:03:32,999] [INFO] [real_accelerator.py:133:get_accele